In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW-2").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [4]:
df_2 = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df_2.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            2|          3|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            2|          2|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [6]:
df_3 = df_2.filter(df_2["total_votes"] >= 20)
df_3.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            8|         31|   N|                Y|
|          5|           21|         21|   N|                Y|
|          3|           59|         69|   N|                Y|
|          1|            6|         28|   N|                Y|
|          5|           21|         21|   N|                Y|
|          4|           27|         32|   N|                Y|
|          5|           34|         40|   N|                Y|
|          5|           30|         30|   N|                Y|
|          5|           17|         20|   N|                Y|
|          4|           57|         65|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [7]:
df_4 = df_3.filter(df_3["helpful_votes"]/df_3["total_votes"] >= 0.5)
df_4.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           21|         21|   N|                Y|
|          3|           59|         69|   N|                Y|
|          5|           21|         21|   N|                Y|
|          4|           27|         32|   N|                Y|
|          5|           34|         40|   N|                Y|
|          5|           30|         30|   N|                Y|
|          5|           17|         20|   N|                Y|
|          4|           57|         65|   N|                Y|
|          5|           19|         20|   N|                Y|
|          5|           18|         20|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [8]:
from pyspark.sql.functions import col, avg
paid_df = df_4.filter(df_4['vine'] == 'Y')
unpaid_df = df_4.filter(df_4['vine'] == 'N')

paid_df.describe().show()
unpaid_df.describe().show()

+-------+------------------+------------------+-----------------+----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+------------------+-----------------+----+-----------------+
|  count|                82|                82|               82|  82|               82|
|   mean|3.8780487804878048|59.207317073170735|66.04878048780488|null|             null|
| stddev| 1.158865387025347| 75.75932290832496| 81.2983990608207|null|             null|
|    min|                 1|                14|               20|   Y|                N|
|    max|                 5|               498|              521|   Y|                Y|
+-------+------------------+------------------+-----------------+----+-----------------+

+-------+------------------+------------------+-----------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes| vine|verified_purchase|
+-------+---------

In [11]:
paid_five_star_reviews = paid_df[paid_df['star_rating'] == 5].count()
total_paid_reviews = paid_df.count()
percentage_five_star_paid_reviews = float(paid_five_star_reviews) / float(total_paid_reviews)
print(total_paid_reviews)
print(paid_five_star_reviews)
print(percentage_five_star_paid_reviews)

82
33
0.4024390243902439


In [12]:
unpaid_five_star_reviews = unpaid_df[unpaid_df['star_rating'] == 5].count()
total_unpaid_reviews = unpaid_df.count()
percentage_five_star_uppaid_reviews = float(unpaid_five_star_reviews) / float(total_unpaid_reviews)
print(total_unpaid_reviews)
print(unpaid_five_star_reviews)
print(percentage_five_star_uppaid_reviews)

24742
12807
0.5176218575701237
